In [1]:
from z3 import *
def term_generator(variables, constant_symbols, unary_operators, sym_bin_operators, asym_bin_operators,depth):
    
    if depth == 0:
        return variables + constant_symbols
    
    else:
        previous = []
        for d in range(0, depth): 
            previous += term_generator(variables, constant_symbols, unary_operators, sym_bin_operators, asym_bin_operators,d)
        new = []
        
        for i in unary_operators:
            for j in previous:
                term = i + "(" + j + ")"
                if is_not_big_constant(term,variables):
                    new.append( term )
                
        for i in asym_bin_operators:
            for j in range(len(previous)):
                for k in range(len(previous)):
                    term = "(" + previous[j] + i + previous[k] +")"
                    if is_not_big_constant(term,variables):
                        new.append( term )
                        
                        
        for i in sym_bin_operators:
            for j in range(len(previous)):
                for k in range(j,len(previous)):
                    term =  "(" + previous[j] + i + previous[k] +")"
                    if is_not_big_constant(term,variables):
                        new.append( term )
                    
        return new + variables + constant_symbols
                    
    
    
def is_not_big_constant(term,variables):
    
    if len(term) == 1:
        return True
    #eiher a small constant (1 symbol) or a variable
    for i in variables:
        if i in term:
            return True
    return False




        

In [23]:
pip install z3-solver
#Should already be installed

SyntaxError: invalid syntax (Temp/ipykernel_23372/4149441412.py, line 1)

In [9]:
sample_terms = term_generator(["x", "y"],  ['1','2'], [] , ["+","*"], ["-"], 1 )
variables = ["x1","y1"]

In [10]:
sample_terms

['(x-x)',
 '(x-y)',
 '(x-1)',
 '(x-2)',
 '(y-x)',
 '(y-y)',
 '(y-1)',
 '(y-2)',
 '(1-x)',
 '(1-y)',
 '(2-x)',
 '(2-y)',
 '(x+x)',
 '(x+y)',
 '(x+1)',
 '(x+2)',
 '(y+y)',
 '(y+1)',
 '(y+2)',
 '(x*x)',
 '(x*y)',
 '(x*1)',
 '(x*2)',
 '(y*y)',
 '(y*1)',
 '(y*2)',
 'x',
 'y',
 '1',
 '2']

In [21]:
from z3 import *

def collapse_terms(sample_terms):
    new_terms = []
    
    
    for i in range(len(sample_terms)):
        simplified_term = str(simplify(eval(sample_terms[i])))
        if simplified_term not in new_terms:
            new_terms.append(simplified_term)
            
    return new_terms

def protected_simplify(z3string, variable):
    
    if "/0" not in z3string:
        return str(simplify(eval(z3string)))
    else:
        return variable
    


def proto_test():

    x = Int('x')
    y = Int('y')
    x1 = Int('x1')
    y1 = Int('y1')
    #a = collapse_terms(sample_terms)
    #print(a)

    #print(str(simplify(eval("x + y + 2*x + 3"))))
    solution_found = False
    for i in range(len(sample_terms)):
        for j in range(len(sample_terms)):

            # division by zero is  still a big problem. Temporarily avoiding the issue by removing the division symbol.
            # division by zero raises a z3 exception, which we need to test for in protected_simplify
            # it doesn't work like a boolean or a regular python object, so we can't text for existence/falsity
            s1 = "x1" + "==" + protected_simplify(sample_terms[i],"x1")
            s2 = "y1" + "==" + protected_simplify(sample_terms[j],"y1")



            solver = Solver()
            # (y>0) co (y1=y and x1<=y1)
            solver.add ( y > 0 )
            solver.add (Not(And(y1 == y, x1 <= y1)))
        
            solver.add(eval(s1))
            solver.add(eval(s2)) #plug in here from term generator, 2 lines of the program to be synthesized.
            #         solver.add(y1 == y)
            #         solver.add(x1 <= y1)

            if solver.check() != sat:
                s = 'Solution found with assignment statement \n' + s1 + '\n' + s2
                
                print( s)
                return 1
                #solution_found = True
                
    print("No solutions found")
    return(0)
        

proto_test()    
# y > 0 , x = 2*y, x > y (end is negated due to validity testing)
# should return No solution found

Solution found with assignment statement 
x1==0
y1==y


1